In [10]:
!git clone https://github.com/mateoKutnjak/pose_est.git
import sys
sys.path.append('/content/pose_est')

fatal: destination path 'pose_est' already exists and is not an empty directory.


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!ls "/content/drive/My Drive"

'1. Pretraživanja.gslides'
 annotations
'Colab Notebooks'
 CV-Europass-20190209-EN-2.pdf
 Europass
 images
'Napoleon Hill - Think and Grow Rich - Unabridged Version'
 src.zip


In [0]:
from pose_est.dataset import MPII_dataset
from pose_est.models import HourglassModel
import tensorflow as tf
import keras.backend as k
import argparse

In [0]:
IMAGES_DIR = '/content/drive/My Drive/images'
ANNOTATIONS_JSON_FILE = '/content/drive/My Drive/annotations/annotations.json'
INPUT_SHAPE = (256, 256, 3)
OUTPUT_SHAPE = (64, 64, 16)
BATCH_SIZE = 8
STACKS_NUM = 2

In [0]:
mpii_dataset = MPII_dataset(
        images_dir=IMAGES_DIR,
        annots_json_filename=ANNOTATIONS_JSON_FILE,
        input_shape=INPUT_SHAPE,
        output_shape=OUTPUT_SHAPE,
        type='train'
    )

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0

In [0]:
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
hg_model = HourglassModel(
    dataset=mpii_dataset,
    batch_size=BATCH_SIZE,
    stacks_num=STACKS_NUM,
)

In [26]:
hg_model.build()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
front_module_conv2d_7x7 (Conv2D (None, 128, 128, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_103 (BatchN (None, 128, 128, 64) 256         front_module_conv2d_7x7[0][0]    
__________________________________________________________________________________________________
front_residual_1_conv_1x1_first (None, 128, 128, 64) 4160        batch_normalization_103[0][0]    
__________________________________________________________________________________________________
batch_norm

In [0]:
hg_model.train()

A
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
